# DML: Tenecia de armas y tasas de homicidios

Cook y Ludwig (2006) estudian el efecto de la tenencia de armas sobre la prevalencia de tasas de homicidios. Ellos estiman una elasiticad de la tasa de homicidios respecto a la tenencia en el orden de 0.1 a 0.3, lo que se traslada a un costo marginal social de la tenencia de armas en el orden de US$100 a US$1800. Debido a que la tenencia de armas es una variable no observada, los autores usan la fracción de suicidios cometidos con arma de fuego como proxy de la tenencia de armas. 

Se usan datos para 195 condados en el periodo 1980 a 1990. Los autores estiman la siguiente ecuación

\begin{equation*}
log Y_{it}=\beta_0+\beta_1log FSS_{it-1}+\beta_2X_{it}+d_i+d_t+\epsilon_{it}
\end{equation*}

donde $X_{it}$ es un conjunto de variables de control, $d_i$ y $d_t$ son efectos fijos por condado y año

## Cargar y procesar datos

In [81]:
import pandas as pd
import numpy as np
import re
import warnings
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import KFold, cross_val_predict
warnings.simplefilter('ignore')



In [82]:
file = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/gun_clean.csv"
data = pd.read_csv(file)
data.shape

(3900, 415)

In [83]:
data.head()

,Unnamed: 0,CountyCode,logfssl,BPS030D,BPS130D,BPS230D,BNK010D,BNK050D,AGE010D,AGE050D,...,X_Jfips191,X_Jfips192,X_Jfips193,X_Jfips194,X_Jfips195,newblack,newfhh,newmove,newdens,newmal
0,1,1073,3.572171,6.558784,10.335562,6.38017,4.051633,12.209261,10.907922,2.775915,...,0,0,0,0,0,2.287224,1.538038,3.550657,3.089752,0.966946
1,2,1073,3.549750,6.558784,10.335562,6.38017,4.051633,12.209261,10.907922,2.775915,...,0,0,0,0,0,2.287224,1.538038,3.550657,3.089752,0.966946
2,3,1073,3.555511,6.558784,10.335562,6.38017,4.051633,12.209261,10.907922,2.775915,...,0,0,0,0,0,2.287224,1.538038,3.550657,3.089752,0.966946
3,4,1073,3.579848,6.558784,10.335562,6.38017,4.051633,12.209261,10.907922,2.775915,...,0,0,0,0,0,2.287224,1.538038,3.550657,3.089752,0.966946
4,5,1073,3.539653,6.558784,10.335562,6.38017,4.051633,12.209261,10.907922,2.775915,...,0,0,0,0,0,2.287224,1.538038,3.550657,3.089752,0.966946


In [ ]:
# Lista de variables en el dataset
print("📊 VARIABLES EN EL DATASET:")
print("=" * 50)
print(f"Número total de variables: {len(data.columns)}")
print(f"Número de observaciones: {len(data)}")
print("\n📋 Lista de variables:")
for i, col in enumerate(data.columns, 1):
    print(f"{i:2d}. {col}")

print("\n🔍 INFORMACIÓN DETALLADA:")
print("=" * 50)
data.info()

In [84]:
# County FE
county_vars = data.filter(like='X_J')

# Time variables and population weights
# Pull out time variables
time_vars = data.filter(like='X_T')

# Use these to construct population weights
popweights = time_vars.sum(axis=1)

# Unweighted time variables
time_vars = time_vars.div(popweights, axis=0)

# For any columns with only zeros, drop them
time_vars = time_vars.loc[:, (time_vars != 0).any(axis=0)]

# Create time index
time_ind = (time_vars * np.arange(1, 21)).sum(axis=1)

In [85]:
def varlist(df=None, type=["numeric", "factor", "character"], pattern="", exclude=None):
    vars = []
    if any(t in type for t in ["numeric", "factor", "character"]):
        if "numeric" in type:
            vars += df.select_dtypes(include=["number"]).columns.tolist()
        if "factor" in type:
            vars += df.select_dtypes(include=["category"]).columns.tolist()
        if "character" in type:
            vars += df.select_dtypes(include=["object"]).columns.tolist()

    if exclude:
        vars = [var for var in vars if var not in exclude]

    if pattern:
        vars = [var for var in vars if re.search(pattern, var)]

    return vars

In [86]:
census = []
census_var = ["^AGE", "^BN", "^BP", "^BZ", "^ED", "^EL", "^HI", "^HS",
              "^INC", "^LF", "^LN", "^PI", "^PO", "^PP", "^PV", "^SPR", "^VS"]

for pattern in census_var:
    census += varlist(df=data, pattern=pattern)

X1 = ["logrobr", "logburg", "burg_missing", "robrate_missing"]
X2 = ["newblack", "newfhh", "newmove", "newdens", "newmal"]

# "Treatment" variable
d = "logfssl"

# Outcome variable
y = "logghomr"

# New DataFrame for time index
usedata = pd.DataFrame({'time.ind': time_ind})
usedata['weights'] = popweights

varlist_all = [y, d] + X1 + X2 + census
for var in varlist_all:
    usedata[var] = data[var]

# Construct initial conditions
varlist0 = [y] + X1 + X2 + census
for var in varlist0:
    usedata[f'{var}0'] = np.kron(usedata.loc[usedata['time.ind'] == 1, var], np.ones(20))

# County means
county_vars = pd.DataFrame(county_vars)
for var in varlist_all:
    usedata[f'{var}J'] = county_vars.dot(np.linalg.pinv(county_vars).dot(usedata[var]))

# Time means
time_vars = pd.DataFrame(time_vars)
for var in varlist_all:
    usedata[f'{var}T'] = time_vars.dot(np.linalg.pinv(time_vars).dot(usedata[var]))


## Estimaciones



In [87]:
# Regresión base

X=sm.add_constant(usedata['logfssl'])
y=usedata['logghomr']
lm0 = sm.OLS(y, X).fit(cov_type='HC3')
vc0 = lm0.cov_params()
coef = lm0.params['logfssl']
std_err = np.sqrt(vc0.loc['logfssl', 'logfssl'])
print("Base:", coef, "(", std_err, ")")

Base: 0.30202337690461406 ( 0.012565715795207547 )


In [88]:
# Regresión con efectos fijos por año y condado

county_vars_clean = county_vars.iloc[:,1:]  # Excluir la primera columna para evitar multicolinealidad
Xfe = pd.concat([X, county_vars_clean, time_vars], axis=1)

In [89]:
lm1 = sm.OLS(y, Xfe).fit(cov_type='HC3')
vc1 = lm1.cov_params()
coef = lm1.params['logfssl']
std_err = np.sqrt(vc1.loc['logfssl', 'logfssl'])
print("Con efectos fijos:", coef, "(", std_err, ")")

Con efectos fijos: 0.2886642634053608 ( 0.06127564276162746 )


In [90]:
# Modelo con controles base

Xc =pd.concat([X,usedata[X1+X2+census]],axis=1)
lmC = sm.OLS(y, Xc).fit(cov_type='HC3')
vcC = lmC.cov_params()
coef = lmC.params['logfssl']
std_err = np.sqrt(vcC.loc['logfssl', 'logfssl'])
print("OLS with Controls:", coef, "(", std_err, ")")

OLS with Controls: 0.31027647478489406 ( 0.07301925169951773 )


In [91]:
#Modelo con controles base y promedios temporales y por condado

varlistX = X1 + X2 + census
varlistMeans = [d] + X1 + X2 + census

# Adding county and time means
for var in varlistX:
    varlistMeans.append(var + 'J')
    varlistMeans.append(var + 'T')

Xcm = sm.add_constant(usedata[varlistMeans])

In [92]:
mM = sm.OLS(y, Xcm).fit(cov_type='HC3')
vcM = mM.cov_params()
coef = mM.params['logfssl']
std_err = np.sqrt(vcM.loc['logfssl', 'logfssl'])
print("OLS with Averages:", coef, "(", std_err, ")")

OLS with Averages: 0.16784091766043352 ( 0.6120103474928204 )


In [93]:
# Modelo con todos los controles

Xcf = sm.add_constant(usedata.drop(columns=['logghomr']))
y = usedata['logghomr']
lmA = sm.OLS(y, Xcf).fit(cov_type='HC3')
vcA = lmA.cov_params()
coef = lmA.params['logfssl']
std_err = np.sqrt(vcA.loc['logfssl', 'logfssl'])
print("OLS All:", coef, "(", std_err, ")")

OLS All: 0.1793159014234836 ( 0.06668125805040237 )


## Estimaciones DML

In [94]:
#Definimos la función para el crossfitting
def dml(X, D, y, modely, modeld, *, nfolds, classifier=False):
    '''
    DML for the Partially Linear Model setting with cross-fitting

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely: the ML model for predicting the outcome y
    modeld: the ML model for predicting the treatment D
    nfolds: the number of folds in cross-fitting
    classifier: bool, whether the modeld is a classifier or a regressor

    time: array of time indices, eg [0,1,...,T-1,0,1,...,T-1,...,0,1,...,T-1]
    clu: array of cluster indices, eg [1073, 1073, 1073, ..., 5055, 5055, 5055, 5055]
    cluster: bool, whether to use clustered standard errors

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    Dhat: the cross-fitted predictions for the treatment D
    resy: the outcome residuals
    resD: the treatment residuals
    epsilon: the final residual-on-residual OLS regression residual
    '''
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)  # shuffled k-folds
    yhat = cross_val_predict(modely, X, y, cv=cv, n_jobs=-1)  # out-of-fold predictions for y
    # out-of-fold predictions for D
    # use predict or predict_proba dependent on classifier or regressor for D
    if classifier:
        Dhat = cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
    else:
        Dhat = cross_val_predict(modeld, X, D, cv=cv, n_jobs=-1)
    # calculate outcome and treatment residuals
    resy = y - yhat
    resD = D - Dhat

    dml_data = pd.concat([pd.Series(resy, name='resy'), pd.Series(resD, name='resD')], axis=1)
    ols_mod = smf.ols(formula='resy ~ 1 + resD', data=dml_data).fit()

    point = ols_mod.params[1]
    stderr = ols_mod.bse[1]
    epsilon = ols_mod.resid

    return point, stderr, yhat, Dhat, resy, resD, epsilon


In [95]:
# Función para resumen de resultados
def summary(point, stderr, yhat, Dhat, resy, resD, epsilon, X, D, y, *, name):
    '''
    Convenience summary function that takes the results of the DML function
    and summarizes several estimation quantities and performance metrics.
    '''
    return pd.DataFrame({'estimate': point,  # point estimate
                         'stderr': stderr,  # standard error
                         'lower': point - 1.96 * stderr,  # lower end of 95% confidence interval
                         'upper': point + 1.96 * stderr,  # upper end of 95% confidence interval
                         'rmse y': np.sqrt(np.mean(resy**2)),  # RMSE of model that predicts outcome y
                         'rmse D': np.sqrt(np.mean(resD**2))  # RMSE of model that predicts treatment D
                         }, index=[name])

In [96]:
#OLS básico para learners

from sklearn.linear_model import LinearRegression


Y = usedata['logghomr']
D = usedata['logfssl']
Z = pd.DataFrame({"Const": np.ones(len(Y))})  # regression on constant

modely = make_pipeline(StandardScaler(), LinearRegression())
modeld = make_pipeline(StandardScaler(), LinearRegression())

# Run DML model with nfolds folds of cross-fitting
result_OLS = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = summary(*result_OLS, Z, D, Y, name='OLS No Controls')
table

,estimate,stderr,lower,upper,rmse y,rmse D
OLS No Controls,0.30196,0.013676,0.275155,0.328764,1.096108,1.210306


In [97]:
# Lasso para learners

cv = KFold(n_splits=10, shuffle=True, random_state=123)  # shuffled k-folds


In [98]:
varlistX = X1 + X2 + census
varlistMeans = [d] + X1 + X2 + census

# Adding county and time means
for var in varlistX:
    varlistMeans.append(var + 'J')
    varlistMeans.append(var + 'T')

Z = sm.add_constant(usedata[varlistMeans].drop(columns=['logfssl']))

In [99]:
modely = make_pipeline(StandardScaler(), LassoCV(cv=cv))
modeld = make_pipeline(StandardScaler(), LassoCV(cv=cv))

# Run DML model with nfolds folds of cross-fitting
result_LassoCV = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_LassoCV, Z, D, y, name='LassoCV')])
table

,estimate,stderr,lower,upper,rmse y,rmse D
OLS No Controls,0.301960,0.013676,0.275155,0.328764,1.096108,1.210306
LassoCV,0.517538,0.057333,0.405165,0.629911,0.531980,0.147088


In [100]:
# RF como learners

from sklearn.ensemble import RandomForestRegressor


modely = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, min_samples_leaf=5, random_state=123))
modeld = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, min_samples_leaf=5, random_state=123))

# Run DML model with nfolds folds of cross-fitting (computationally intensive)
result_RF = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_RF, Z, D, Y, name='RF')])
table

,estimate,stderr,lower,upper,rmse y,rmse D
OLS No Controls,0.301960,0.013676,0.275155,0.328764,1.096108,1.210306
LassoCV,0.517538,0.057333,0.405165,0.629911,0.531980,0.147088
RF,0.099402,0.053464,-0.005387,0.204191,0.412544,0.123541
